In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


In [2]:
 #set the size of our plots as they are a little small by default.
plt.rcParams["figure.figsize"] = (20,5)

In [3]:
df = pd.read_csv(r'/home/user/Data Science Projects/Taxi/Predicting-car-insurance-cost-throughcollision-risk-through-machine-learning-/LBD_New_York_collisions_and_weather_data.csv')
df.head()

day  year  mo  da collision_date  temp  dewp     slp  visib  wdsp  mxpsd  \
0    2  2013   1   1     01/01/2013  37.8  23.6  1011.9   10.0   6.1    8.9   
1    3  2013   1   2     02/01/2013  27.1  10.5  1016.8   10.0   5.3    9.9   
2    4  2013   1   3     03/01/2013  28.4  14.1  1020.6   10.0   3.7    8.0   
3    5  2013   1   4     04/01/2013  33.4  18.6  1017.0   10.0   6.5   13.0   
4    6  2013   1   5     05/01/2013  36.1  18.7  1020.6   10.0   6.6   12.0   

   gust   max   min  prcp   sndp  fog  NUM_COLLISIONS  
0  19.0  39.9  33.1   0.0  999.9    0             381  
1  19.0  33.1  21.9   0.0  999.9    0             480  
2  15.0  32.0  24.1   0.0  999.9    0             549  
3  24.1  37.0  30.0   0.0  999.9    0             505  
4  21.0  42.1  32.0   0.0  999.9    0             389

In [4]:
df = df.sort_values(["year", "mo", "da"], ascending = (True, True, True)) # order the data by year, month, day in ascending order.
df.head() # check the data again by viewing the first 5 rows

day  year  mo  da collision_date  temp  dewp     slp  visib  wdsp  mxpsd  \
0    2  2013   1   1     01/01/2013  37.8  23.6  1011.9   10.0   6.1    8.9   
1    3  2013   1   2     02/01/2013  27.1  10.5  1016.8   10.0   5.3    9.9   
2    4  2013   1   3     03/01/2013  28.4  14.1  1020.6   10.0   3.7    8.0   
3    5  2013   1   4     04/01/2013  33.4  18.6  1017.0   10.0   6.5   13.0   
4    6  2013   1   5     05/01/2013  36.1  18.7  1020.6   10.0   6.6   12.0   

   gust   max   min  prcp   sndp  fog  NUM_COLLISIONS  
0  19.0  39.9  33.1   0.0  999.9    0             381  
1  19.0  33.1  21.9   0.0  999.9    0             480  
2  15.0  32.0  24.1   0.0  999.9    0             549  
3  24.1  37.0  30.0   0.0  999.9    0             505  
4  21.0  42.1  32.0   0.0  999.9    0             389

In [5]:
total_collisions = df['NUM_COLLISIONS'].sum()
print(type(total_collisions))
print("Total collisions before cleaning:", total_collisions)


<class 'numpy.int64'>
Total collisions before cleaning: 1319249


Cleaning


In [6]:
#original df
original_df = df

df_cleaned = df[(df["NUM_COLLISIONS"] > 350) & (df["NUM_COLLISIONS"] < 900)]
df_cleaned = df_cleaned.dropna(how='all')
df_cleaned.dropna(subset=['NUM_COLLISIONS'], inplace=True) 
df_cleaned['collision_date'] = pd.to_numeric(df['collision_date'], errors='coerce')  
total_collisions_post_clean = df_cleaned['NUM_COLLISIONS'].sum()
print("Total collisions before cleaning:", total_collisions)
print("Total total_collisions_post_clean:", total_collisions_post_clean)
difference = total_collisions - total_collisions_post_clean
print(f"Dropped {difference} rows")


Total collisions before cleaning: 1319249
Total total_collisions_post_clean: 1307273
Dropped 11976 rows


Descriptive statistics

In [7]:
def collision_grouping_decorator(group_by):
    """Decorator to group collisions by day or month."""
    def wrapper(func):
        def inner(dataframe):
            collisions = dataframe.groupby(group_by)['NUM_COLLISIONS'].sum().reset_index()
            collisions.columns = ['Group', 'Total Collisions']
            return collisions.sort_values(by='Total Collisions', ascending=False)
        return inner
    return wrapper

@collision_grouping_decorator('day')
def group_collisions_by_day(dataframe):
    """Groups collisions by day of the week."""
    pass  # The actual logic is handled by the decorator

@collision_grouping_decorator('mo')
def group_collisions_by_month(dataframe):
    """Groups collisions by month."""
    pass  # The actual logic is handled by the decorator

# Get collisions grouped by day and month
collisions_per_day_df = group_collisions_by_day(df_cleaned)
collisions_per_month_df = group_collisions_by_month(df_cleaned)

# Print results
print("Collisions per Day:")
print(collisions_per_day_df)

print("\nCollisions per Month:")
print(collisions_per_month_df)


##Interesting - Friday as the most collisions, with Sunday the least - rush hour, commuting
## so what's the connection between Friday and Sunday. Sunday is a quieter day, Friday busier. Don't have a time of day so can't look at this in more detail.
##is there time data available?


#Correlation between weather? Rainfall on a particular day? or by month?

Collisions per Day:
   Group  Total Collisions
4      5            207693
3      4            196751
1      2            194514
2      3            194134
0      1            187203
5      6            175254
6      7            151724

Collisions per Month:
    Group  Total Collisions
5       6            118319
9      10            116505
4       5            116190
6       7            113076
8       9            111650
7       8            111309
10     11            109884
11     12            108360
2       3            106432
3       4            104204
0       1             99241
1       2             92103


In [8]:
#examine weather data etc



In [9]:
# Grouping by 'year' and summing 'NUM_COLLISIONS' for each year
collisions_per_year = df_cleaned.groupby('year')['NUM_COLLISIONS'].sum()

# Convert the series to a DataFrame for plotting
collisions_per_year_df = collisions_per_year.reset_index()

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(collisions_per_year_df['year'], collisions_per_year_df['NUM_COLLISIONS'], color='skyblue')
plt.xlabel('Year')
plt.ylabel('Number of Collisions')
plt.title('Number of Collisions Per Year')
plt.xticks(collisions_per_year_df['year'])  # Ensure all years are displayed on the x-axis
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Narrative - ok clearly collisions are increasing, year on year

In [10]:
#Narrative - I am going to examine whether the number of collisions for each day of the week changes from year to year

# Define a mapping for days of the week
day_mapping = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}

# Apply the day mapping to the 'day' column
df_cleaned.loc[:, 'day_mapped'] = df_cleaned['day'].map(day_mapping)

# Group by year and day of the week, then sum the number of collisions
collisions_per_day_per_year = df_cleaned.groupby(['year', 'day_mapped'])['NUM_COLLISIONS'].sum().reset_index()

# Pivot the table to have years as rows and days of the week as columns
collisions_per_day_per_year_pivot = collisions_per_day_per_year.pivot_table(index='year', columns='day_mapped', values='NUM_COLLISIONS', fill_value=0)

# Display the resulting table
print(collisions_per_day_per_year_pivot)


day_mapped   Friday   Monday  Saturday   Sunday  Thursday  Tuesday  Wednesday
year                                                                         
2013        33109.0  29453.0   27702.0  24135.0   29726.0  30214.0    29138.0
2014        32415.0  29003.0   27658.0  22833.0   30473.0  29628.0    30586.0
2015        33146.0  31059.0   27978.0  24644.0   32906.0  32496.0    32403.0
2016        37147.0  32000.0   30794.0  26059.0   34682.0  34314.0    34235.0
2017        34948.0  32716.0   30640.0  27464.0   34445.0  33721.0    33856.0
2018        36928.0  32972.0   30482.0  26589.0   34519.0  34141.0    33916.0


Regression Analysis

In [11]:
df_2012 = df_cleaned[df_cleaned["year"] == 2012]
df_2012.loc[df_2012['day'] > 0, 'day'] = df_2012['day']+1 # change all days by adding 1.

df_2012.loc[df_2012['day'] == 8, 'day'] = 1 # change days that equal 8 to day 1

In [12]:
# Filter data for the year 2013
df_2013 = df_cleaned[df_cleaned["year"] == 2013].copy()

# Increment all days by 1
df_2013.loc[df_2013['day'] > 0, 'day'] += 1

# Change days that are equal to 8 to 1
df_2013.loc[df_2013['day'] == 8, 'day'] = 1


# Filter data for the year 2014
df_2014 = df_cleaned[df_cleaned["year"] == 2014].copy()

# Increment all days by 1
df_2014.loc[df_2014['day'] > 0, 'day'] += 1

# Change days that are equal to 8 to 1
df_2014.loc[df_2014['day'] == 8, 'day'] = 1

# Filter data for the year 2015
df_2015 = df_cleaned[df_cleaned["year"] == 2015].copy()

# Increment all days by 1
df_2015.loc[df_2015['day'] > 0, 'day'] += 1

# Change days that are equal to 8 to 1
df_2015.loc[df_2015['day'] == 8, 'day'] = 1

# Filter data for the year 2016
df_2016 = df_cleaned[df_cleaned["year"] == 2016].copy()

# Increment all days by 1
df_2016.loc[df_2016['day'] > 0, 'day'] += 1

# Change days that are equal to 8 to 1
df_2016.loc[df_2016['day'] == 8, 'day'] = 1

In [13]:
all_years = [df_2013, df_2014, df_2015, df_2016]
df_final = pd.concat(all_years)
print(df_final["day"].count())

df_final.head()
print ("ran this!")

df_final.describe()


1446
ran this!


day        year           mo           da  collision_date  \
count  1446.000000  1446.00000  1446.000000  1446.000000             0.0   
mean      4.018672  2014.50000     6.538036    15.736515             NaN   
std       1.995929     1.12151     3.429710     8.794879             NaN   
min       1.000000  2013.00000     1.000000     1.000000             NaN   
25%       2.000000  2013.00000     4.000000     8.000000             NaN   
50%       4.000000  2014.50000     7.000000    16.000000             NaN   
75%       6.000000  2016.00000    10.000000    23.000000             NaN   
max       7.000000  2016.00000    12.000000    31.000000             NaN   

              temp         dewp          slp        visib         wdsp  \
count  1446.000000  1446.000000  1446.000000  1446.000000  1446.000000   
mean     55.856570    40.460235  1141.416321    28.152420    16.337690   
std      17.562679    19.170412  1049.480490   136.606818   107.333302   
min       9.800000   -11.800000   992.100000     1.700000     0.900000   
25%      41.900000    26.225000  1012.700000     8.500000     3.300000   
50%      57.700000    42.050000  1017.000000     9.800000     4.400000   
75%      72.075000    56.375000  1021.875000    10.000000     5.800000   
max      89.100000    74.800000  9999.900000   999.900000   999.900000   

             mxpsd         gust          max          min         prcp  \
count  1446.000000  1446.000000  1446.000000  1446.000000  1446.000000   
mean     21.072752   344.061342    65.375864    48.035408     0.132801   
std     106.841720   461.040350    18.307613    17.194228     0.350377   
min       4.100000    14.000000    17.600000     1.900000     0.000000   
25%       7.000000    18.100000    50.275000    35.100000     0.000000   
50%       8.900000    22.900000    66.900000    48.900000     0.000000   
75%      11.100000   999.900000    82.000000    64.000000     0.070000   
max     999.900000   999.900000    98.100000    82.900000     4.530000   

              sndp          fog  NUM_COLLISIONS  
count  1446.000000  1446.000000     1446.000000  
mean    910.673306     0.071231      587.784232  
std     283.992694     0.257299       92.945349  
min       1.200000     0.000000      355.000000  
25%     999.900000     0.000000      521.000000  
50%     999.900000     0.000000      589.000000  
75%     999.900000     0.000000      650.000000  
max     999.900000     1.000000      872.000000

In [14]:
# check out this plot
groups = df_final.groupby('year') # We group by year as we want to create a legend and make the visualization clearer using color.
plt.ylim(0, 1480)

for name, group in groups:
    plt.plot(group.da, group.NUM_COLLISIONS, marker='o', linestyle='', markersize=2, label=name)

plt.legend()

# Adding axis titles
plt.xlabel('Day of the Year')
plt.ylabel('Number of Collisions')

plt.show()

Regression model

In [15]:
import tensorflow as tf
import keras
from tensorflow import keras
from tensorflow.keras import layers
from keras import layers
from tensorflow.python.keras.layers import Dense
print(tf.__version__)



2024-10-20 17:12:44.442415: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-20 17:12:45.269783: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-20 17:12:45.743087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 17:12:46.240570: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 17:12:46.382390: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 17:12:47.238426: I tensorflow/core/platform/cpu_feature_gu

2.17.0


In [16]:
#very simple model, with just one single input (days) and one output (NUM_COLLISIONS).

one_input_data = [df_final["day"], df_final["NUM_COLLISIONS"]] # create an array of all values for day and all values for NUM_TRIPS in two columns
headers = ["day", "NUM_COLLISIONS"] # declare the titles of our input and output. As you can see day is first and NUM_TRIPS is second and they correspond to the line above
df_one_input = pd.concat(one_input_data, axis=1, keys=headers) # Bring these two arrays together to make a new dataframe
df_one_input.head()

day  NUM_COLLISIONS
0    3             381
1    4             480
2    5             549
3    6             505
4    7             389

In [17]:
 #ok now to train model
 #This code is performing a common task in data science: splitting a dataset into training and test sets. 

train_dataset = df_one_input.sample(frac=0.8, random_state=0)
test_dataset = df_one_input.drop(train_dataset.index)



In [18]:
train_dataset.describe()

day  NUM_COLLISIONS
count  1157.000000     1157.000000
mean      3.994814      586.856525
std       1.996963       92.742613
min       1.000000      355.000000
25%       2.000000      519.000000
50%       4.000000      589.000000
75%       6.000000      649.000000
max       7.000000      872.000000

In [19]:
#test_dataset.describe()

day  NUM_COLLISIONS
count  289.000000      289.000000
mean     4.114187      591.498270
std      1.992374       93.822384
min      1.000000      357.000000
25%      2.000000      526.000000
50%      4.000000      590.000000
75%      6.000000      653.000000
max      7.000000      867.000000

In [39]:
#train_features = train_dataset.copy(): Creates a copy of the train_dataset DataFrame and stores it in train_features.
#test_features = test_dataset.copy(): Similarly, creates a copy of the test_dataset DataFrame and stores it in test_features.
#Why .copy()?: The .copy() method is used to ensure that the original train_dataset and test_dataset are not modified when changes are made to train_features and test_features.

train_features = train_dataset.copy()
test_features = test_dataset.copy()

#The labels (train_labels and test_labels) represent the actual number of collisions in each row of your dataset. 
# The model will use other features (input variables) to try and predict this value. 

train_labels = train_features.pop('NUM_COLLISIONS')
test_labels = test_features.pop('NUM_COLLISIONS')



In [21]:
print(train_features.head())
print(test_features.head())

      day
1096    7
482     2
184     5
302     4
995     4
    day
11    7
23    5
24    6
25    7
28    3


In [22]:
scale_factor = 1000000

In [23]:
train_labels = train_labels/scale_factor
test_labels = test_labels/scale_factor
print(train_labels)

##train_labels = train_labels / scale_factor: This line divides each element in the train_labels Series by the scale_factor (which is set to 1,000,000). 
# This effectively scales down the values of the labels in the training dataset, making them smaller and potentially easier to work with.
#test_labels = test_labels / scale_factor: 
# Similarly, this line divides each element in the test_labels Series by the same scale_factor, scaling down the labels in the testing dataset.

1096    0.000422
482     0.000597
184     0.000438
302     0.000558
995     0.000501
          ...   
460     0.000484
589     0.000572
147     0.000686
160     0.000624
1124    0.000596
Name: NUM_COLLISIONS, Length: 1157, dtype: float64


In [24]:
import tensorflow as tf
import numpy as np

# Assuming train_features is your training data

#tf.keras.layers.Normalization: This creates a normalization layer that will standardize the features (input data).
#axis=-1: This indicates that normalization will be applied across the last axis. 
# For example, if you have a 2D array (matrix), normalization will be applied to each row independently.

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

first = np.array(train_features[:1])

# Print the first example before and after normalization
with np.printoptions(precision=2, suppress=True):
    print('First example:', first)
    print()
    print('Normalized:', normalizer(first).numpy())


I0000 00:00:1729440789.914367   32550 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-20 17:13:09.916374: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


First example: [[7]]

Normalized: [[1.51]]


In [25]:
import tensorflow as tf
import numpy as np

# Assuming train_features is your DataFrame and 'day' is one of its columns
day = np.array(train_features['day'])

# train_features['day']: This selects the column labeled 'day' from the train_features DataFrame. 
# It assumes that train_features is a pandas DataFrame containing various features.

# tf.keras.layers.Normalization: This creates a normalization layer specifically for the day feature.
#input_shape=[1,]: This indicates that the input to the layer will be a 1D array with one element (the day value).
#axis=None: This means that normalization will be applied to the entire input (single value), rather than along a specific axis.

# Define and adapt the Normalization layer
day_normalizer = tf.keras.layers.Normalization(input_shape=[1,], axis=None)
day_normalizer.adapt(day)


/home/user/.pyenv/versions/3.11.1/lib/python3.11/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [26]:
import tensorflow as tf
import numpy as np

# This creates a sequential model, which is a linear stack of layers. 
# You can think of it as a chain where the output of one layer is the input to the next.

# Assuming day_normalizer is already defined as in your previous code
# Define the model
day_model = tf.keras.Sequential([
    day_normalizer,
    tf.keras.layers.Dense(units=1)  # accessing Dense layer through tf.keras.layers
])

# : This adds a Dense layer with a single unit (or neuron). 
# A Dense layer is a fully connected layer where each input node is connected to each output node. 
# The output will be a single value, which might represent a prediction (for example, predicting a target variable based on the normalized day).

# Print model summary
day_model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_1 (Normalization) │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5 (24.00 B)

 Trainable params: 2 (8.00 B)

 Non-trainable params: 3 (16.00 B)

In [27]:
day_model.predict(day[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


array([[ 0.06249394],
       [-0.04148289],
       [ 0.02090321],
       [ 0.00010784],
       [ 0.00010784],
       [ 0.00010784],
       [-0.02068752],
       [ 0.04169857],
       [-0.02068752],
       [ 0.06249394]], dtype=float32)

In [28]:
day_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

print("It worked")

It worked


Keras model - exec training for 100 epochs

In [29]:
%%time
history = day_model.fit(
    train_features['day'],
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

CPU times: user 6.62 s, sys: 503 ms, total: 7.12 s
Wall time: 6.43 s


In [30]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

loss  val_loss  epoch
95  0.014770  0.017662     95
96  0.009145  0.010744     96
97  0.014513  0.011068     97
98  0.016242  0.019797     98
99  0.017207  0.004514     99

In [31]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 0.2])
  plt.xlabel('Epoch')
  plt.ylabel('Error [NUM_TRIPS]')
  plt.legend()
  plt.grid(True)

In [32]:
plot_loss(history)

In [33]:
test_results = {}

test_results['day_model'] = day_model.evaluate(
    test_features['day'],
    test_labels, verbose=0)

In [34]:
x = tf.linspace(1, 7, 8)
y = day_model.predict(x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [35]:
#sense check
print("Unique days:", train_features['day'].unique())
print("Number of trips:", train_labels.shape)


Unique days: [7 2 5 4 3 6 1]
Number of trips: (1157,)


In [36]:
print("Predicted days:", x[:10])
print("Predicted NUM_TRIPS:", y[:10])


Predicted days: tf.Tensor(
[1.         1.85714286 2.71428571 3.57142857 4.42857143 5.28571429
 6.14285714 7.        ], shape=(8,), dtype=float64)
Predicted NUM_TRIPS: [[0.00927511]
 [0.00802116]
 [0.0067672 ]
 [0.00551325]
 [0.00425929]
 [0.00300534]
 [0.00175139]
 [0.00049743]]


In [37]:
plt.boxplot(train_labels)  # Boxplot to check for outliers
plt.show()


In [38]:
def plot_day(x, y):
  plt.scatter(train_features['day'], train_labels, label='Data', alpha=1)
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('day')
  plt.ylim([0.2, 0.8])
  plt.ylabel('NUM_TRIPS')
  plt.legend()

plot_day(x, y)